In [1]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [2]:
from googleapiclient.discovery import build
from datetime import datetime
import json

# API Key (gunakan environment variable di production!)
API_KEY = "your_api_key"
youtube = build('youtube', 'v3', developerKey=API_KEY)

class YouTubeChannelAnalyzer:
    def __init__(self, api_key):
        self.youtube = build('youtube', 'v3', developerKey=api_key)

    def get_channel_info(self, channel_identifier):
        """
        Dapatkan informasi channel berdasarkan:
        - Channel ID (UCxxxxx)
        - Username (@username)
        - Channel URL
        """
        try:
            # Coba dengan channel ID dulu
            if channel_identifier.startswith('UC') or channel_identifier.startswith('@'):
                if channel_identifier.startswith('@'):
                    # Handle @username format
                    request = self.youtube.channels().list(
                        part="snippet,statistics,contentDetails",
                        forHandle=channel_identifier
                    )
                else:
                    # Handle channel ID
                    request = self.youtube.channels().list(
                        part="snippet,statistics,contentDetails",
                        id=channel_identifier
                    )
            else:
                # Coba cari berdasarkan nama channel
                search_request = self.youtube.search().list(
                    q=channel_identifier,
                    part="snippet",
                    type="channel",
                    maxResults=1
                )
                search_response = search_request.execute()

                if not search_response['items']:
                    print(f"❌ Channel '{channel_identifier}' tidak ditemukan")
                    return None

                channel_id = search_response['items'][0]['id']['channelId']
                request = self.youtube.channels().list(
                    part="snippet,statistics,contentDetails",
                    id=channel_id
                )

            response = request.execute()

            if response['items']:
                return response['items'][0]
            else:
                print(f"❌ Channel '{channel_identifier}' tidak ditemukan")
                return None

        except Exception as e:
            print(f"❌ Error: {e}")
            return None

    def display_channel_info(self, channel_data):
        """Tampilkan informasi channel"""
        snippet = channel_data['snippet']
        stats = channel_data['statistics']

        print("🏆 INFORMASI CHANNEL")
        print("=" * 50)
        print(f"📺 Nama: {snippet['title']}")
        print(f"📝 Deskripsi: {snippet['description'][:200]}...")
        print(f"📅 Dibuat: {snippet['publishedAt'][:10]}")
        print(f"🏠 Negara: {snippet.get('country', 'Tidak diketahui')}")
        print(f"👥 Subscriber: {int(stats['subscriberCount']):,}")
        print(f"📹 Total Video: {stats['videoCount']}")
        print(f"👀 Total Views: {int(stats['viewCount']):,}")
        print("=" * 50)

    def get_channel_videos(self, channel_id, max_results=50):
        """Dapatkan semua video dari channel"""
        try:
            # Dapatkan uploads playlist ID
            channel_response = self.youtube.channels().list(
                part="contentDetails",
                id=channel_id
            ).execute()

            uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

            videos = []
            next_page_token = None

            while len(videos) < max_results:
                # Dapatkan video dari playlist uploads
                playlist_request = self.youtube.playlistItems().list(
                    part="snippet",
                    playlistId=uploads_playlist_id,
                    maxResults=min(50, max_results - len(videos)),
                    pageToken=next_page_token
                )
                playlist_response = playlist_request.execute()

                # Extract video IDs
                video_ids = [item['snippet']['resourceId']['videoId'] for item in playlist_response['items']]

                # Dapatkan detail video (views, likes, dll)
                videos_request = self.youtube.videos().list(
                    part="snippet,statistics",
                    id=','.join(video_ids)
                )
                videos_response = videos_request.execute()

                videos.extend(videos_response['items'])

                next_page_token = playlist_response.get('nextPageToken')
                if not next_page_token:
                    break

            return videos[:max_results]

        except Exception as e:
            print(f"❌ Error mengambil video: {e}")
            return []

    def analyze_channel_history(self, channel_identifier, max_videos=50):
        """Analisis lengkap channel dan historynya"""
        print(f"🔍 Menganalisis channel: {channel_identifier}\n")

        # Dapatkan info channel
        channel_data = self.get_channel_info(channel_identifier)
        if not channel_data:
            return

        channel_id = channel_data['id']
        self.display_channel_info(channel_data)

        # Dapatkan video history
        print(f"\n📹 MENGAMBIL {max_videos} VIDEO TERBARU...")
        videos = self.get_channel_videos(channel_id, max_videos)

        if not videos:
            print("❌ Tidak ada video ditemukan")
            return

        print(f"✅ Berhasil mengambil {len(videos)} video\n")

        # Analisis video
        self.display_video_history(videos)
        self.analyze_performance(videos)

        return {
            'channel_info': channel_data,
            'videos': videos
        }

    def display_video_history(self, videos):
        """Tampilkan history video"""
        print("📋 HISTORY VIDEO TERBARU")
        print("=" * 80)

        for i, video in enumerate(videos[:20], 1):  # Tampilkan 20 teratas
            snippet = video['snippet']
            stats = video['statistics']

            title = snippet['title'][:60] + "..." if len(snippet['title']) > 60 else snippet['title']
            views = int(stats.get('viewCount', 0))
            likes = int(stats.get('likeCount', 0))
            published = snippet['publishedAt'][:10]

            print(f"{i:2d}. {title}")
            print(f"    📅 {published} | 👀 {views:,} views | 👍 {likes:,} likes")
            print(f"    🔗 https://youtu.be/{video['id']}")
            print()

    def analyze_performance(self, videos):
        """Analisis performa video"""
        if not videos:
            return

        print("\n📊 ANALISIS PERFORMA")
        print("=" * 50)

        # Hitung statistik
        total_views = sum(int(v['statistics'].get('viewCount', 0)) for v in videos)
        total_likes = sum(int(v['statistics'].get('likeCount', 0)) for v in videos)
        avg_views = total_views / len(videos)
        avg_likes = total_likes / len(videos)

        # Video terpopuler
        most_viewed = max(videos, key=lambda x: int(x['statistics'].get('viewCount', 0)))
        most_liked = max(videos, key=lambda x: int(x['statistics'].get('likeCount', 0)))

        print(f"📈 Rata-rata views per video: {avg_views:,.0f}")
        print(f"👍 Rata-rata likes per video: {avg_likes:,.0f}")
        print(f"🏆 Video terpopuler: {most_viewed['snippet']['title'][:50]}...")
        print(f"    👀 {int(most_viewed['statistics'].get('viewCount', 0)):,} views")
        print(f"❤️ Video paling disukai: {most_liked['snippet']['title'][:50]}...")
        print(f"    👍 {int(most_liked['statistics'].get('likeCount', 0)):,} likes")

        # Analisis trend upload
        self.analyze_upload_trend(videos)

    def analyze_upload_trend(self, videos):
        """Analisis trend upload"""
        from collections import Counter

        # Hitung upload per bulan
        upload_months = []
        for video in videos:
            date = datetime.fromisoformat(video['snippet']['publishedAt'].replace('Z', '+00:00'))
            month_year = f"{date.year}-{date.month:02d}"
            upload_months.append(month_year)

        month_counts = Counter(upload_months)

        print(f"\n📅 TREND UPLOAD (dari {len(videos)} video terakhir)")
        print("-" * 30)
        for month, count in sorted(month_counts.items(), reverse=True)[:6]:
            print(f"{month}: {count} video")

# Fungsi utama untuk analisis channel
def analyze_youtube_channel(channel_name, max_videos=50):
    """Fungsi utama untuk menganalisis channel YouTube"""
    analyzer = YouTubeChannelAnalyzer(API_KEY)
    return analyzer.analyze_channel_history(channel_name, max_videos)

# Contoh penggunaan
if __name__ == "__main__":
    # Contoh 1: Analisis channel berdasarkan nama
    print("🚀 CONTOH 1: Analisis Channel PewDiePie")
    analyze_youtube_channel("PewDiePie", max_videos=30)

    print("\n" + "="*100 + "\n")

    # Contoh 2: Analisis channel berdasarkan @username
    print("🚀 CONTOH 2: Analisis Channel MrBeast")
    analyze_youtube_channel("@MrBeast", max_videos=20)

    print("\n" + "="*100 + "\n")

    # Contoh 3: Analisis channel lokal Indonesia
    print("🚀 CONTOH 3: Analisis Channel Indonesia")
    analyze_youtube_channel("Atta Halilintar", max_videos=25)

# Fungsi tambahan untuk channel spesifik
def search_and_analyze_channel(search_term, max_videos=30):
    """Cari dan analisis channel berdasarkan kata kunci"""
    analyzer = YouTubeChannelAnalyzer(API_KEY)

    # Cari channel berdasarkan keyword
    try:
        search_request = analyzer.youtube.search().list(
            q=search_term,
            part="snippet",
            type="channel",
            maxResults=5
        )
        search_response = search_request.execute()

        if search_response['items']:
            print(f"🔍 Ditemukan {len(search_response['items'])} channel untuk '{search_term}':")
            for i, channel in enumerate(search_response['items'], 1):
                print(f"{i}. {channel['snippet']['title']} - {channel['snippet']['channelTitle']}")

            # Analisis channel pertama
            first_channel_id = search_response['items'][0]['id']['channelId']
            print(f"\n📊 Menganalisis channel pertama...\n")
            return analyzer.analyze_channel_history(first_channel_id, max_videos)

    except Exception as e:
        print(f"❌ Error: {e}")

# Fungsi untuk mendapatkan trending videos dari channel
def get_trending_videos_from_channel(channel_name, days_back=30):
    """Dapatkan video trending dari channel dalam periode tertentu"""
    analyzer = YouTubeChannelAnalyzer(API_KEY)

    channel_data = analyzer.get_channel_info(channel_name)
    if not channel_data:
        return

    videos = analyzer.get_channel_videos(channel_data['id'], max_results=100)

    # Filter video dalam periode tertentu
    from datetime import datetime, timedelta
    cutoff_date = datetime.now() - timedelta(days=days_back)

    recent_videos = []
    for video in videos:
        video_date = datetime.fromisoformat(video['snippet']['publishedAt'].replace('Z', '+00:00'))
        if video_date.replace(tzinfo=None) > cutoff_date:
            recent_videos.append(video)

    # Sort berdasarkan views
    trending_videos = sorted(recent_videos,
                           key=lambda x: int(x['statistics'].get('viewCount', 0)),
                           reverse=True)

    print(f"🔥 TOP TRENDING VIDEOS dari {channel_name} ({days_back} hari terakhir)")
    print("=" * 70)

    for i, video in enumerate(trending_videos[:10], 1):
        title = video['snippet']['title'][:50] + "..." if len(video['snippet']['title']) > 50 else video['snippet']['title']
        views = int(video['statistics'].get('viewCount', 0))
        print(f"{i:2d}. {title}")
        print(f"    👀 {views:,} views | 🔗 https://youtu.be/{video['id']}")
        print()

# Contoh penggunaan khusus
print("\n🎯 CONTOH PENGGUNAAN KHUSUS:")
print("1. analyze_youtube_channel('Nama Channel', max_videos=50)")
print("2. search_and_analyze_channel('keyword pencarian')")
print("3. get_trending_videos_from_channel('Nama Channel', days_back=7)")

🚀 CONTOH 1: Analisis Channel PewDiePie
🔍 Menganalisis channel: PewDiePie

🏆 INFORMASI CHANNEL
📺 Nama: PewDiePie
📝 Deskripsi: I make videos....
📅 Dibuat: 2010-04-29
🏠 Negara: JP
👥 Subscriber: 110,000,000
📹 Total Video: 4817
👀 Total Views: 29,552,496,411

📹 MENGAMBIL 30 VIDEO TERBARU...
✅ Berhasil mengambil 30 video

📋 HISTORY VIDEO TERBARU
 1. This Trip Was NOT What We Planned
    📅 2025-06-15 | 👀 305,444 views | 👍 25,605 likes
    🔗 https://youtu.be/4uqo-brP3xs

 2. Presenting my Billion Dollar Plan...
    📅 2025-05-28 | 👀 2,791,868 views | 👍 211,098 likes
    🔗 https://youtu.be/pgeTa1PV_40

 3. Cycle across Japan!
    📅 2025-05-16 | 👀 1,143,727 views | 👍 64,250 likes
    🔗 https://youtu.be/qGXtMENJDps

 4. I installed Linux (so should you)
    📅 2025-04-26 | 👀 5,900,772 views | 👍 299,464 likes
    🔗 https://youtu.be/pVI_smLgTY0

 5. We bought Land in Japan
    📅 2025-04-11 | 👀 3,841,323 views | 👍 185,173 likes
    🔗 https://youtu.be/tMIDpbmFoL8

 6. 30 Day Dream Challenge.
    📅 2025-

In [3]:
from googleapiclient.discovery import build
from datetime import datetime
import json

# API Key (gunakan environment variable di production!)
API_KEY = "your_api_key"

class YouTubeAnalyzer:
    def __init__(self, api_key):
        self.youtube = build('youtube', 'v3', developerKey=api_key)

    def get_videos_by_channel(self, channel_identifier, max_results=20):
        """
        Ambil video dari channel berdasarkan:
        - Channel ID (UCxxxxx)
        - Username (@username)
        - Nama channel
        """
        try:
            # Dapatkan channel info dulu
            channel_data = self._get_channel_info(channel_identifier)
            if not channel_data:
                return None

            channel_id = channel_data['id']
            channel_name = channel_data['snippet']['title']

            # Dapatkan uploads playlist ID
            uploads_playlist_id = channel_data['contentDetails']['relatedPlaylists']['uploads']

            videos = []
            next_page_token = None

            while len(videos) < max_results:
                # Ambil video dari playlist uploads
                playlist_request = self.youtube.playlistItems().list(
                    part="snippet",
                    playlistId=uploads_playlist_id,
                    maxResults=min(50, max_results - len(videos)),
                    pageToken=next_page_token
                )
                playlist_response = playlist_request.execute()

                # Extract video IDs
                video_ids = [item['snippet']['resourceId']['videoId'] for item in playlist_response['items']]

                # Dapatkan detail video
                videos_request = self.youtube.videos().list(
                    part="snippet,statistics",
                    id=','.join(video_ids)
                )
                videos_response = videos_request.execute()

                videos.extend(videos_response['items'])

                next_page_token = playlist_response.get('nextPageToken')
                if not next_page_token:
                    break

            print(f"✅ Berhasil mengambil {len(videos)} video dari channel: {channel_name}")
            return videos[:max_results]

        except Exception as e:
            print(f"❌ Error: {e}")
            return None

    def get_videos_by_keyword(self, keyword, max_results=20):
        """
        Cari video berdasarkan keyword
        """
        try:
            videos = []
            next_page_token = None

            while len(videos) < max_results:
                # Search videos by keyword
                search_request = self.youtube.search().list(
                    q=keyword,
                    part="snippet",
                    type="video",
                    maxResults=min(50, max_results - len(videos)),
                    order="relevance",
                    pageToken=next_page_token
                )
                search_response = search_request.execute()

                # Extract video IDs
                video_ids = [item['id']['videoId'] for item in search_response['items']]

                # Dapatkan detail video
                videos_request = self.youtube.videos().list(
                    part="snippet,statistics",
                    id=','.join(video_ids)
                )
                videos_response = videos_request.execute()

                videos.extend(videos_response['items'])

                next_page_token = search_response.get('nextPageToken')
                if not next_page_token:
                    break

            print(f"✅ Berhasil mencari {len(videos)} video dengan keyword: '{keyword}'")
            return videos[:max_results]

        except Exception as e:
            print(f"❌ Error: {e}")
            return None

    def get_video_by_id(self, video_id):
        """
        Ambil detail video berdasarkan video ID
        """
        try:
            # Bersihkan video ID jika berupa URL
            if 'youtu.be/' in video_id:
                video_id = video_id.split('youtu.be/')[-1].split('?')[0]
            elif 'youtube.com/watch?v=' in video_id:
                video_id = video_id.split('v=')[-1].split('&')[0]

            # Ambil detail video
            video_request = self.youtube.videos().list(
                part="snippet,statistics,contentDetails",
                id=video_id
            )
            video_response = video_request.execute()

            if video_response['items']:
                video = video_response['items'][0]
                print(f"✅ Berhasil mengambil video: {video['snippet']['title']}")
                return video
            else:
                print(f"❌ Video dengan ID '{video_id}' tidak ditemukan")
                return None

        except Exception as e:
            print(f"❌ Error: {e}")
            return None

    def _get_channel_info(self, channel_identifier):
        """Helper function untuk mendapatkan info channel"""
        try:
            if channel_identifier.startswith('UC'):
                # Channel ID
                request = self.youtube.channels().list(
                    part="snippet,contentDetails",
                    id=channel_identifier
                )
            elif channel_identifier.startswith('@'):
                # Handle @username
                request = self.youtube.channels().list(
                    part="snippet,contentDetails",
                    forHandle=channel_identifier
                )
            else:
                # Cari berdasarkan nama channel
                search_request = self.youtube.search().list(
                    q=channel_identifier,
                    part="snippet",
                    type="channel",
                    maxResults=1
                )
                search_response = search_request.execute()

                if not search_response['items']:
                    print(f"❌ Channel '{channel_identifier}' tidak ditemukan")
                    return None

                channel_id = search_response['items'][0]['id']['channelId']
                request = self.youtube.channels().list(
                    part="snippet,contentDetails",
                    id=channel_id
                )

            response = request.execute()
            return response['items'][0] if response['items'] else None

        except Exception as e:
            print(f"❌ Error: {e}")
            return None

    def display_videos(self, videos, title="VIDEO LIST"):
        """Tampilkan daftar video dengan format yang rapi"""
        if not videos:
            print("❌ Tidak ada video untuk ditampilkan")
            return

        print(f"\n📹 {title}")
        print("=" * 80)

        for i, video in enumerate(videos, 1):
            snippet = video['snippet']
            stats = video['statistics']

            title_text = snippet['title'][:60] + "..." if len(snippet['title']) > 60 else snippet['title']
            views = int(stats.get('viewCount', 0))
            likes = int(stats.get('likeCount', 0))
            published = snippet['publishedAt'][:10]
            channel = snippet['channelTitle']

            print(f"{i:2d}. {title_text}")
            print(f"    📺 {channel} | 📅 {published}")
            print(f"    👀 {views:,} views | 👍 {likes:,} likes")
            print(f"    🔗 https://youtu.be/{video['id']}")
            print()

    def display_single_video(self, video):
        """Tampilkan detail video tunggal"""
        if not video:
            print("❌ Tidak ada video untuk ditampilkan")
            return

        snippet = video['snippet']
        stats = video['statistics']

        print(f"\n🎬 DETAIL VIDEO")
        print("=" * 50)
        print(f"📺 Judul: {snippet['title']}")
        print(f"👤 Channel: {snippet['channelTitle']}")
        print(f"📅 Dipublikasi: {snippet['publishedAt'][:10]}")
        print(f"👀 Views: {int(stats.get('viewCount', 0)):,}")
        print(f"👍 Likes: {int(stats.get('likeCount', 0)):,}")
        print(f"💬 Comments: {int(stats.get('commentCount', 0)):,}")
        print(f"🔗 URL: https://youtu.be/{video['id']}")
        print(f"📝 Deskripsi: {snippet['description'][:200]}...")


# Inisialisasi analyzer
analyzer = YouTubeAnalyzer(API_KEY)

# Fungsi-fungsi utama yang mudah digunakan
def get_channel_videos(channel_name, max_videos=20):
    """Ambil video dari channel"""
    videos = analyzer.get_videos_by_channel(channel_name, max_videos)
    if videos:
        analyzer.display_videos(videos, f"VIDEO DARI CHANNEL: {channel_name}")
    return videos

def search_videos(keyword, max_videos=20):
    """Cari video berdasarkan keyword"""
    videos = analyzer.get_videos_by_keyword(keyword, max_videos)
    if videos:
        analyzer.display_videos(videos, f"HASIL PENCARIAN: {keyword}")
    return videos

def get_video_details(video_id):
    """Ambil detail video berdasarkan ID"""
    video = analyzer.get_video_by_id(video_id)
    if video:
        analyzer.display_single_video(video)
    return video


# Contoh penggunaan
if __name__ == "__main__":
    print("🚀 YOUTUBE ANALYZER - VERSI MINIMAL")
    print("=" * 50)

    # Contoh 1: Ambil video dari channel
    print("\n1️⃣ AMBIL VIDEO DARI CHANNEL")
    get_channel_videos("@MrBeast", max_videos=5)

    # Contoh 2: Cari video berdasarkan keyword
    print("\n2️⃣ CARI VIDEO BERDASARKAN KEYWORD")
    search_videos("python tutorial", max_videos=5)

    # Contoh 3: Ambil detail video berdasarkan ID
    print("\n3️⃣ DETAIL VIDEO BERDASARKAN ID")
    get_video_details("dQw4w9WgXcQ")  # Never Gonna Give You Up

    print("\n✨ CARA PENGGUNAAN:")
    print("• get_channel_videos('nama_channel', max_videos=20)")
    print("• search_videos('keyword', max_videos=20)")
    print("• get_video_details('video_id_atau_url')")

🚀 YOUTUBE ANALYZER - VERSI MINIMAL

1️⃣ AMBIL VIDEO DARI CHANNEL
✅ Berhasil mengambil 5 video dari channel: MrBeast

📹 VIDEO DARI CHANNEL: @MrBeast
 1. Feastables vs Reese’s Taste Test with @gordonramsay
    📺 MrBeast | 📅 2025-06-10
    👀 126,227,338 views | 👍 2,867,929 likes
    🔗 https://youtu.be/ZGvn9ywXjpY

 2. $1 vs $500,000 Romantic Date
    📺 MrBeast | 📅 2025-06-07
    👀 74,198,788 views | 👍 2,475,615 likes
    🔗 https://youtu.be/hTSaweR8qMI

 3. Gordon Ramsay Decides Feastables vs Hershey’s
    📺 MrBeast | 📅 2025-06-06
    👀 27,702,991 views | 👍 891,248 likes
    🔗 https://youtu.be/g2nwnC1Xn0E

 4. Can I Teach a Penguin To Subscribe?
    📺 MrBeast | 📅 2025-05-31
    👀 112,899,343 views | 👍 2,717,265 likes
    🔗 https://youtu.be/KCRLP-zBacU

 5. Extreme Helmet Game
    📺 MrBeast | 📅 2025-05-28
    👀 117,190,988 views | 👍 2,492,548 likes
    🔗 https://youtu.be/c2ukL_zjFCk


2️⃣ CARI VIDEO BERDASARKAN KEYWORD
✅ Berhasil mencari 5 video dengan keyword: 'python tutorial'

📹 HASIL PE